### **Motion detection**
from https://pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/

This article will detail how to build a basic motion detection and tracking system using computer vision techniques. This example will work with both pre-recorded videos and live streams from your webcam. This system will be developped or laptops/desktops.

**Background subtraction intro**

Background subtraction is critical in many computer vision applications. We use it to count the number of cars passing through a toll booth. We use it to count the number of people walking in and out of a store. *And we use it for motion detection.*

Before we get started coding in this post, let me say that there are many, many ways to perform motion detection, tracking, and analysis in OpenCV:
- cv2.BackgroundSubtractorMOG function. See http://www.ee.surrey.ac.uk/CVSSP/Publications/papers/KaewTraKulPong-AVBS01.pdf
- cv2.BackgroundSubtractorMOG2 function. See ...

Newer version of openCV have probability based foreground and background segmentation (see https://goldberg.berkeley.edu/pubs/acc-2012-visual-tracking-final.pdf) through the cv2.createBackgroundSubtractorGMG function in openCV 3.

In motion detection, we tend to make the following assumption:

*The background of our video stream is largely static and unchanging over consecutive frames of a video. Therefore, if we can model the background,if there is a substantial change, we can detect it — this change normally corresponds to **motion** on our video.*

In the real-world this assumption can easily fail. Due to shadowing, reflections, lighting conditions, and any other possible change in the environment, our background can look quite different in various frames of a video. The most successful background subtraction/foreground detection systems utilize fixed mounted cameras and controlled lighting conditions.

The methods explained above while effective, can also be computationally expensive. We'll keep it simple.

In [ ]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", help="path to the video file")
ap.add_argument("-a", "--min-area", type=int, default=500, help="minimum area size")
args = vars(ap.parse_args())
# if the video argument is None, then we are reading from webcam
if args.get("video", None) is None:
	vs = VideoStream(src=0).start()
	time.sleep(2.0)
# otherwise, we are reading from a video file
else:
	vs = cv2.VideoCapture(args["video"])
# initialize the first frame in the video stream
firstFrame = None

- **--video**, is optional. It simply defines a path to a pre-recorded video file that we can detect motion in. If you do not supply a path to a video file, then OpenCV will utilize your webcam to detect motion.
- **--min-area**, which is the minimum size (in pixels) for a region of an image to be considered actual “motion”. We’ll often find small regions of an image that have changed substantially, likely due to noise or changes in lighting conditions so we’ll define a minimum size of a region to combat and filter out these false-positives.

We’ll grab a reference to the webcam and wait for it to warm up. And if a video file is supplied, then we’ll create a pointer to it.

*Assumption:* **The first frame of our video file will contain no motion and just background — therefore, we can model the background of our video stream using only the first frame of the video.**

In [ ]:
# loop over the frames of the video
while True:
	# grab the current frame and initialize the occupied/unoccupied
	# text
	frame = vs.read()
	frame = frame if args.get("video", None) is None else frame[1]
	text = "Unoccupied"
	# if the frame could not be grabbed, then we have reached the end
	# of the video
	if frame is None:
		break
	# resize the frame, convert it to grayscale, and blur it
	frame = imutils.resize(frame, width=500)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	gray = cv2.GaussianBlur(gray, (21, 21), 0)
	# if the first frame is None, initialize it
	if firstFrame is None:
		firstFrame = gray
		continue